# Creating Lat Lon variable for the CISM grid
This notebook creates 2D Latitude and Longitude variables from the cartesian grid coordinate of a CISM grid file. The Cartesian grid can be generated using the Jupyter notebook "Create_input_file_CISM.ipynb".

Author: gunterl@ucar.edu

In [2]:
# Import packages
import sys, os
sys.path.insert(1, '../lib/')

import constants
import utils
import numpy as np
from locate_vectorize import locate
from netCDF4 import Dataset
from pathlib import Path


# to display figures in notebook after executing the code.
%matplotlib inline 

In [3]:
# Configuration
IS = 'GrIS'  # Ice sheet for which we are creating the grid
res = 4000  # grid resolution (CISM has a uniform grid)
str_res = str(res).zfill(5)

Rearth = constants.Rearth  # Earth's radius (m)
ecc = constants.ecc  # eccentricity

# Path and file name of the CISM grid file
pathlocal = os.getcwd()
grid_path = f"{pathlocal}/cism_grid/"
grid_file = f"{grid_path}xy_CISM3_grid_{IS}_{str_res}.nc"

# Destination directory and file name to write the Lat-Lon file. 
dstDir = f"{pathlocal}/LatLon/{IS}/"
dstFile = f"{dstDir}CISM3_initialization_{IS}_{str_res}.LatLon.nc"

# Creating the path where the Lat-Lon file will be written if it does not exist
Path(dstDir).mkdir(parents=True, exist_ok=True)

In [4]:
# Loading the Cartesian CISM grid data information
nid = Dataset(grid_file, 'r')
x1_dst = nid.variables['x1'][:]
y1_dst = nid.variables['y1'][:]
x0_dst = nid.variables['x0'][:]
y0_dst = nid.variables['y0'][:]
nid.close()

nx1_dst = len(x1_dst)
ny1_dst = len(y1_dst)
nx0_dst = len(x0_dst)
ny0_dst = len(y0_dst)

print('nx1_dst and ny1_dst =', nx1_dst, ny1_dst)
print('nx0_dst and ny0_dst =', nx0_dst, ny0_dst)

nx1_dst and ny1_dst = 421 721
nx0_dst and ny0_dst = 420 720


In [5]:
# Converting x-y to lat-lon

Lat1_2d_dst, Lon1_2d_dst = utils.create_latlon_coord(x1_dst, y1_dst, IS, Rearth, ecc)
Lat0_2d_dst, Lon0_2d_dst = utils.create_latlon_coord(x0_dst, y0_dst, IS, Rearth, ecc)

In [6]:
# print(Lon1_2d_dst[0:10,0])
# print(Lon1_2d_dst[0:10,-1])

In [7]:
# Writing the data to file

if os.path.isfile(dstFile):
    os.remove(dstFile)


###############
# Output file #
###############

# Creating the netcdf output file
ncfile = Dataset(dstFile, 'w')

# Create dimensions
ncfile.createDimension('x1', nx1_dst)
ncfile.createDimension('y1', ny1_dst)
ncfile.createDimension('x0', nx0_dst)
ncfile.createDimension('y0', ny0_dst)

x1           = ncfile.createVariable('x1','f4',('x1',))
x1.long_name = "Cartesian x-coordinate collocated"
x1.units     = "m"
x1[:] = x1_dst[:]

y1           = ncfile.createVariable('y1','f4',('y1',))
y1.long_name = "Cartesian y-coordinate collocated"
y1.units     = "m"
y1[:] = y1_dst[:]

x0           = ncfile.createVariable('x0','f4',('x0',))
x0.long_name = "Cartesian staggered velocity x-coordinate"
x0.units     = "m"
x0[:] = x0_dst[:]

y0           = ncfile.createVariable('y0','f4',('y0',))
y0.long_name = "Cartesian staggered velocity y-coordinate"
y0.units     = "m"
y0[:] = y0_dst[:]

Lat1           = ncfile.createVariable('Lat1','f4',('y1','x1'))
Lat1.long_name = "Latitude"
Lat1.units     = "Degree North"
Lat1.comment   = "Latitude with respect to the thickness grid"
Lat1[:,:] = Lat1_2d_dst[:,:]

Lon1           = ncfile.createVariable('Lon1','f4',('y1','x1'))
Lon1.long_name = "Longitude"
Lon1.units     = "Degree East"
Lon1.comment   = "Longitude with respect to the thickness grid"
Lon1[:,:] = Lon1_2d_dst[:,:]

Lat0           = ncfile.createVariable('Lat0','f4',('y0','x0'))
Lat0.long_name = "Latitude"
Lat0.units     = "Degree North"
Lat0.comment   = "Latitude with respect to the velocity grid"
Lat0[:,:] = Lat0_2d_dst[:,:]

Lon0           = ncfile.createVariable('Lon0','f4',('y0','x0'))
Lon0.long_name = "Longitude"
Lon0.units     = "Degree East"
Lon0.comment   = "Longitude with respect to the velocity grid"
Lon0[:,:] = Lon0_2d_dst[:,:]

# Global attributes
ncfile.author = 'Gunter Leguy'
ncfile.Rearth = f"{Rearth} (m)"
ncfile.eccentricity = f"{ecc}"
ncfile.close()

In [ ]:
del Lon0_2d_dst, Lat0_2d_dst, Lon1_2d_dst, Lat1_2d_dst, x1_dst, y1_dst, x0_dst, y0_dst